---
# Object Detection/Localizator
---
## Tasks Outline:

#### Idea: start with the most basic task, gradually make it more complex 
* Step 1) identify actual boxes
* Step 2) locate an actual object (a Pokemon)
* Step 3) different sized objects
* Step 4) flipped objects
* Step 5) add actual backgrounds 
* Step 6) object may not appear 
* Step 7) multiple classes of objects

---
---
# Step 1) identify actual boxes
---
---